# Taller 01: Métodos de Representación

In [ ]:
!pip install gensim

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

El dataset fue obtenido de Kaggle: https://www.kaggle.com/datasets/exactful/wikipedia-movies/data

In [ ]:
df = pd.read_csv('1990s-movies.csv')
df.shape

In [ ]:
df['plot'].head()

## Preprocesamiento
Antes de poder trabajar con el texto, debemos preprocesarlo, de forma que aseguremos que el input que ingrese a nuestros análisis sea el correcto.

### 1. Pasar todo a minúsculas

In [ ]:
df['plot'] = df['plot'].str.lower()
df['plot'].head()

### 2. Eliminar los espacios en blanco en exceso

In [ ]:
" ".join("hola      mi    nombre  es erasmo".split())

In [ ]:
df['plot'] = df['plot'].apply(lambda x: " ".join(x.split()))
df['plot'].head()

### 3. Tokenizamos las palabras
Nota: En algunos casos puede ser necesario tokenizar primero por oraciones y luego por palabras.

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

df['plot'] = df['plot'].apply(word_tokenize)
df['plot'].head()

### 3. Remover los stopwords
Los stopwords son palabras que se usan con frecuencia, pero no aportan al texto en muchos casos.

In [ ]:
from nltk.corpus import stopwords

en_stopwords = stopwords.words('english')
df['plot'] = df['plot'].apply(lambda x: [y for y in x if y not in en_stopwords])
df['plot'].head()

### 4. Remover signos de puntuación
Se eliminan los signos de puntuación, ya que en la mayoría de casos no tienen aporte al significado.

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r"\w+")
df['plot'] = df['plot'].apply(lambda x: tokenizer.tokenize(' '.join(x)))
df['plot'].head()

### 7. Guardamos la forma no tokenizada
Para algunos algoritmos, requeriremos la forma no tokenizadas; mientras que para otros sí. Por tanto, los guardamos en columnas separadas.

In [ ]:
df['plot2']= df['plot'].apply(lambda x: ' '.join(x))

In [ ]:
df[['plot','plot2']].head()

# Representación Bag of Words
Se puede obtener la representación de Bag of Words fácilmente empleando el CountVectorizer de sklearn. Cabe destacar que esta función tiene algunos argumentos de preproceamiento, por lo que algunos pasos que se ejecutaron previamente estarían duplicados. Revise la documentación en: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
# Se genera un dataset de 2 documentos, solo para visualizar
df_disperso = df[:10].copy(deep=True)
df_disperso

In [ ]:
df_disperso['plot2']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#BAG OF WORDS
vectorizer = CountVectorizer() #Se crea primero
vectorizer.fit(df_disperso['plot2'])

In [ ]:
# Obtenemos el vocabulario para saber qué índice corresponde a cada palabra
vocabulary = vectorizer.vocabulary_
vocabulary = {k: v for k, v in sorted(vocabulary.items(), key=lambda item: item[1])}
print("Vocabulary: ", vocabulary)

In [ ]:
len(vocabulary)

In [ ]:
df_disperso['plot2'][0]

In [ ]:
df_disperso.iloc[0][['plot2']]

Ahora veremos un caso y lo transformaremos a su forma de Bag of Words

In [ ]:
# El primer 1 del vector corresponde a la posicion 9. La palabra en esa posición es addiction.
vector = vectorizer.transform(df_disperso.iloc[0][['plot2']])
print("Encoded Document is:")
print(vector.toarray())

In [ ]:
len(vector.toarray()[0])

In [ ]:
len(vector.toarray()[0])

In [ ]:
# Corroboramos si la palabra addiction se encuentra en el texto
[x for x in df_disperso.iloc[0]['plot2'].split(' ') if x=='addiction']

In [ ]:
vector.toarray()[0]

# Introducción a Word2Vec

**Word2Vec** es una técnica computacional que convierte palabras en vectores de números, representando el significado y relaciones entre ellas. Si dos palabras aparecen en contextos similares, sus vectores también serán similares.

### Ejemplo práctico con gensim

In [ ]:
# Si no tienes gensim instalado, ejecuta esto:
!pip install gensim

In [ ]:
# Corpus sencillo tokenizado
corpus = [
    ["el", "gato", "se", "sentó", "en", "la", "alfombra"],
    ["el", "perro", "se", "echó", "en", "la", "alfombra"],
    ["la", "reina", "y", "el", "rey", "viven", "en", "el", "castillo"],
    ["el", "gato", "y", "el", "perro", "son", "amigos"],
    ["la", "princesa", "es", "hija", "de", "la", "reina"],
    ["la", "reina", "es", "una","mujer"],
    ["el", "rey", "es", "un","hombre"],
    ["el", "gato", "come", "en","el", "castillo"],
    ["el", "gato", "duerme", "en","el", "castillo"],
    ["el", "perro", "come", "en","el", "castillo"],
    ["el", "perro", "duerme", "en","el", "castillo"],
    ["el", "gato", "come", "en","el", "castillo"],
    ["el", "rey", "come", "en","el", "castillo"],
    ["el", "rey", "duerme", "en","el", "castillo"],
    ["el", "princesa", "come", "en","el", "castillo"],
    ["el", "gato", "come", "en","el", "castillo"],
    ["el", "gato", "duerme", "en","el", "castillo"],
    ["el", "perro", "come", "en","el", "castillo"],
    ["el", "perro", "duerme", "en","el", "castillo"]
]

In [ ]:
from gensim.models import Word2Vec

modelo = Word2Vec(corpus, vector_size=50, window=2, min_count=1, sg=1) #parametros iniciales

### Palabras similares a 'reina'

In [ ]:
similares = modelo.wv.most_similar("gato")
print("Palabras similares a 'gato':")
similares

In [ ]:
for palabra, similitud in similares:
    print(palabra, "->", similitud)

### Analogías: rey - hombre + mujer ≈ reina

In [ ]:
resultado = modelo.wv.most_similar(positive=["rey", "mujer"], negative=["hombre"])
print("Resultado de la analogía rey - hombre + mujer:", resultado[0][0])

### Vector numérico de una palabra ('gato')

In [ ]:
print("Vector de 'gato':")
print(modelo.wv["gato"])

In [ ]:
len(modelo.wv["gato"])

### Visualización de palabras en el espacio semántico
Ahora vamos a graficar cómo Word2Vec ubicó cada palabra en un espacio de 2 dimensiones usando PCA.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

palabras = list(modelo.wv.index_to_key)
vectores = np.array([modelo.wv[palabra] for palabra in palabras])

pca = PCA(n_components=2)
vectores_2d = pca.fit_transform(vectores)

plt.figure(figsize=(10, 7))
plt.scatter(vectores_2d[:, 0], vectores_2d[:, 1])
for i, palabra in enumerate(palabras):
    plt.annotate(palabra, (vectores_2d[i, 0], vectores_2d[i, 1]), fontsize=12)
plt.title("Visualización de palabras en el espacio de Word2Vec (PCA)")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.grid(True)
plt.show()

Este gráfico muestra cómo las palabras se agrupan según su significado/contexto en el espacio de Word2Vec.